# 创建阶段：执行上下文与提升


[ES6 规范中的词法环境与闭包、作用域的关系](./lexical_env.ipynb) 里经常说词法环境是在代码执行时创建的。

但是代码「执行时」到底是什么时候？

接下来我们将拆解代码执行过程，进一步了解代码执行细节。

JS 代码的执行实际上分为两个阶段：创建阶段和执行阶段。

回顾下 [简述 V8 引擎对 JS 代码的处理流程](./src/dive_into_javascript/notebooks/v8_pipeline.ipynb)。

JS 代码如果在加载之后就被执行，那么代码会被解析成完整的 AST 抽象语法树并由 V8 解释器转化成字节码执行，否则代码仅被预解析，预解析仅生成简化的 AST。

因此如果一段代码没有被解析过，需要在创建阶段之前先对代码进行真正的解析。

在创建阶段，V8 会创建一个执行上下文作为代码的执行环境。

执行上下文有两个重要的状态组件：

- 变量环境（Variable Environment）
- 词法环境（Lexical Environment）

注意，执行上下文中的词法环境和 ES6 规范中的词法环境不是一回事。

ES6 规范中的词法环境（Lexical Environment）在 [ES6 规范中的词法环境与闭包、作用域的关系](./lexical_env.ipynb) 里已经介绍过了，
它是一个包含环境记录和对外词法环境引用的结构，是结构。

而执行上下文中的词法环境（LexicalEnvironment）和变量环境是词法环境（Lexical Environment）结构的状态组件，是组件。

并且，词法环境组件和变量环境组件是词法环境结构的。

_词法环境（Lexical Environment）和词法环境（LexicalEnvironment）这两个英文单词，一个有空格，一个没有，有空格的是结构，没空格的是组件_

### 提升

这里插入一个概念：**「提升」**，理解「提升」有利于深入理解执行上下文。

JS 「提升」的意思是变量和函数在实际声明之前就已经存在于其所属作用域。本质是在代码执行的创建阶段就把变量和函数声明记录在执行上下文相应状态组件的环境记录中。

当 V8 执行全局/模块/函数代码的时候，在创建阶段会创建相应的执行上下文。执行上下文初始化阶段创建「变量环境组件」和「词法环境组件」。

在全局/模块/函数作用域里，`var` 变量、函数声明会被添加到「变量环境组件」的环境记录中，`var` 变量在环境记录中被初始化为 `undefined`，函数声明在环境记录中被完整记录，因此在代码执行阶段，`var` 变量在实际声明之前可以被访问，值是 `undefined`，而函数在实际声明之前可以被正常调用。

> 注意区分函数声明和函数表达式，被完整提升的是函数声明 `function foo () {}`，而不是函数表达式 `const foo = function () {}`，函数表达式的提升遵从变量提升规则。

`const` 和 `let` 变量会被添加到「词法环境组件」的环境记录中，但它们并不被初始化，在其所属作用域 `const` 和 `let` 变量实际声明之前，它们无法被访问，这就是「暂时性死区」。

尽管所有的变量和函数都会被「提升」，但由于 `const` 和 `let` 不像 `var` 变量那样会被初始化，它们不能在实际声明之前被访问。

> [[AskJS] how is hoisting significant in javascript?](https://www.reddit.com/r/javascript/comments/u5yf8m/askjs_how_is_hoisting_significant_in_javascript/)


In [6]:
// 变量和函数声明的提升
function foo() {
  console.log(a);
  console.log(sum(1, 2));
  // console.log(b); // ReferenceError: Cannot access 'b' before initialization
  // console.log(c); // ReferenceError: Cannot access 'c' before initialization

  var a = 1;
  const b = 2;
  let c = 3;
  function sum(a: number, b: number) {
    return a + b;
  }
}
foo();


undefined
3


### 块级作用域

不同于全局、模块、函数代码的执行，当 V8 执行代码块的时候并不会在创建阶段创建一个执行上下文，代码块属于其所在的最近的全局/模块/函数作用域的执行上下文。

既然代码块没有自己的执行上下文，那么块级作用域是如何实现的？要知道其他作用域的实现都是通过执行下文内的词法环境结构的状态组件实现的。

上篇文章说过，作用域是通过词法环境实现的，这一点依然正确。先看一下 [ES6 规范对代码块执行过程的描述](https://262.ecma-international.org/6.0/#sec-block-runtime-semantics-evaluation) 和社区关于块级作用域的讨论
[How is block scope managed in the lexical environment?](https://stackoverflow.com/questions/29410249/how-is-block-scope-managed-in-the-lexical-environment)。

翻译一下 ES6 规范描述的代码块的执行过程：

1. 如果代码块为空，返回空。如果代码块有语句列表 `StatementList`，继续执行。
2. 保存当前执行上下文的词法环境组件到 `oldEnv`。
3. 创建一个新的词法环境组件 `blockEnv`，它是 `oldEnv` 的子环境，`blockEnv` 的外部环境引用是 `oldEnv`，`blockEnv` 可以访问 `oldEnv` 的环境记录。
4. 提升 `let`、`const` 变量，实际声明之前不能访问，形成暂时性死区。
5. 将当前执行上下文的词法环境组件替换为 `blockEnv`，现在运行代码块使用的是词法环境 `blockEnv`。
6. 保存语句列表 `StatementList` 的执行结果到 `blockValue` 中。
7. 块执行完毕。将当前执行上下文的词法环境组件恢复为 `oldEnv`。
8. 返回 `blockValue`。

概括一下，当执行 JS 代码块的时候，先保存当前词法环境组件 `oldEnv`，然后创建一个继承 `oldEnv` 的新词法环境组件 `blockEnv`，把当前词法环境组件替换为 `blockEnv`，执行代码块，此时代码块的词法环境组件 `blockEnv` 可以访问代码块外 `oldEnv` 的环境记录，执行完代码块，恢复当前词法环境组件 `blockEnv` 为 `oldEnv`，继续执行代码已经访问不到代码块内部了，因为 `blockEnv` 此时已经不在当前执行上下文的环境记录里了。

通过创建块级词法环境组件，动态的改变代码块外部最近的全局/模块/函数作用域执行上下文的词法环境组件，实现了块级作用域。

> [What is the temporal dead zone?](https://stackoverflow.com/questions/33198849/what-is-the-temporal-dead-zone)


### 块级作用域内的提升

关于块级作用域内的提升，刚才没讲。这里补充。

`var` 变量声明不受块级作用域的限制，也就是说，代码块内声明的 `var` 变量依然会被提升到代码块外的全局/模块/函数作用域。

代码块内的 `const`、`let` 变量仅在块级作用域内被提升，回头看上面代码块的执行过程，当块级词法环境组件创建之后，代码块内的 `const` 和 `let` 即被提升。

代码块内的函数声明比较特别，在非严格模式下，函数声明的提升像 `var`，在严格模式下，函数声明的提升像 `const` 和 `let`。

这有个 [关于提升的测试 demo](../../../samples/hoisting/)。


In [3]:
// 在代码块外可以访问到 var 变量 x，但是无法访问到 let 变量 y
// 说明在函数执行上下文的创建阶段，代码块内的 let 变量并没有并记录到函数执行上下文的词法环境组件中

function bar() {
  console.log(x);
  // console.log(baz()); // ReferenceError: baz is not defined

  // console.log(y); // ReferenceError: y is not defined
  if (true) {
    var x = 1;
    // console.log(y); // ReferenceError: Cannot access 'y' before initialization
    let y = 3;
    function baz() {
      return "baz";
    }
  }
}
bar();


undefined


### 作用域、词法环境、执行上下文三者关系

作用域是一个抽象的概念，表示代码的可访问性和可见性。包括全局、模块、函数和块级作用域。

词法环境是作用域的实现机制。包括全局、函数、模块词法环境。这里的词法环境既可以是词法环境结构也可以是词法环境结构的状态组件。

执行上下文是 JS 代码的执行环境，管理着词法环境结构的状态组件，包括词法环境组件和变量环境组件。
